In [29]:
import ast
import numpy as np
import pandas as pd

from opportunistic_planning import prediction, processing

In [13]:
data = pd.read_csv('task_environments_cooking_list_2022-05-10.csv', header=0)

In [14]:
distances_dict = processing.generate_distances_dict(data, use_string_for_seq=False,
                                                   dimensions=[[2, 'xy']])

In [27]:
results = pd.DataFrame()

for row in range(0, len(data)):
        use_string_for_seq = False
        coordinates = {key: ast.literal_eval(value) for key, value in
                       (elem.split(': ') for elem in data.at[row, 'coordinates'].split(';'))}

        start_coordinates = list(ast.literal_eval(data.at[row, 'start_coordinates']))
        ID = str(data.at[row,'ID'])
        
        if use_string_for_seq == True:
            seq = str(data.at[row, 'sequence'])
            objects = list(data.at[row, 'sequence'])
        else:
            seq = [elem for elem in data.at[row, 'sequence'].split(',')]
            objects = [elem for elem in data.at[row, 'sequence'].split(',')]

        # get list of objects that have relational dependencies, if any (else set to empty list)
        try:
            strong_k = list(data.at[row, 'strong_k'].split(','))
        except AttributeError:
            strong_k = []

        try:
            mid_k = list(data.at[row, 'mid_k'].split(','))
        except AttributeError:
            mid_k = []

        try:
            food_k = list(data.at[row, 'food_k'].split(','))
        except AttributeError:
            food_k = []
            
        k1 = {obj: 1.9 if obj in food_k else 0.2 if obj in strong_k \
              else 0.3 if obj in mid_k else 1.0 for obj in objects}
        c1 = {obj: 1.6 if obj in data.at[row, 'containment'] else 1.0 for obj in objects}
        dim = [2, 'xy']
        
        median = processing.get_median_error('prequential', 
                                             row, ID, objects, coordinates, start_coordinates,
                                             c1, k1, dim, seq, distances_dict, n=100)
        
        results.at[row, 'median'] = median
        results.at[row, 'ID'] = ID

In [28]:
results

,median,ID
0,0.0,s13-d21
1,2.0,s13-d25
2,1.0,s13-d28
3,3.0,s13-d31
4,2.0,s13-d40
...,...,...
119,3.0,s37-d21
120,2.0,s37-d25
121,1.0,s37-d29
122,4.0,s37-d39


In [30]:
np.median(results['median'])

2.0

In [32]:
results.to_csv('results/results_cooking_2D_n100_fitted_to_ts_on_2022-05-16.csv', header=True, index=False)